In [2]:
import pandas as pd
import utils
import plotly.graph_objects as go

In [4]:
def plot_candles(df_plot):
    fig = go.Figure()

    fig.add_trace(go.Candlestick(
        x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
        line=dict(width=1), opacity=1,
        increasing_fillcolor='#24A06B',
        decreasing_fillcolor="#CC2E3C",
        increasing_line_color='#2EC886',  
        decreasing_line_color='#FF3A4C'
    ))

    for c in ['MA_TP', 'BB_UPPER', 'BB_LOWER']:
        fig.add_trace(go.Scatter( 
            x=df_plot.time,
            y=df_plot[c],
            line=dict(width=2,color="#4290f5"),
            line_shape='spline',
            name=c,
            mode='lines'
            ))        

    df_markers = df_plot[df_plot.BB_SIGNAL!=0]
    fig.add_trace(go.Candlestick(
        x=df_markers.time, open=df_markers.mid_o, high=df_markers.mid_h, low=df_markers.mid_l, close=df_markers.mid_c,
        line=dict(width=1), opacity=1,
        increasing_fillcolor='#3480eb',
        decreasing_fillcolor="#3480eb",
        increasing_line_color='#3480eb',  
        decreasing_line_color='#3480eb'
    ))

    fig.update_layout(width=1000,height=400,
        margin=dict(l=10,r=10,b=10,t=10),
        font=dict(size=10,color="#e1e1e1"),
        paper_bgcolor="#1e1e1e",
        plot_bgcolor="#1e1e1e")
    fig.update_xaxes(
        gridcolor="#1f292f",
        showgrid=True,fixedrange=True,rangeslider=dict(visible=False),
        rangebreaks=[
            dict(bounds=["sat", "mon"])
        ]
    )
    fig.update_yaxes(
        gridcolor="#1f292f",
        showgrid=True
    )
    fig.show()

In [12]:
pair = "GBP_USD"
granularity = "H4"
df_raw = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [13]:
cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']

In [14]:
def Get_BB_Signal(row):
    if row.mid_c > row.BB_UPPER and row.mid_c > row.mid_o:
        return -1
    if row.mid_c < row.BB_LOWER and row.mid_c < row.mid_o:
        return 1
    return 0

In [15]:
df = df_raw[cols].copy()
df['TP'] = (df.mid_h + df.mid_l + df.mid_c) / 3
df['MA_TP'] = df.TP.rolling(window=20).mean()
df['STD_TP'] = df.TP.rolling(window=20).std()
df['BB_UPPER'] = df.MA_TP + 2 * df.STD_TP
df['BB_LOWER'] = df.MA_TP - 2 * df.STD_TP
df['BB_SIGNAL'] = df.apply(Get_BB_Signal, axis=1)

In [16]:
df_plot = df.iloc[1000:1150]
plot_candles(df_plot)

In [17]:
df['EMA_LONG'] = df.mid_c.ewm(span=26, min_periods=26).mean()
df['EMA_SHORT'] = df.mid_c.ewm(span=12, min_periods=12).mean()
df['MACD'] = df.EMA_SHORT - df.EMA_LONG
df['SIGNAL'] = df.MACD.ewm(span=9).mean()
df['HIST'] = df.MACD - df.SIGNAL
df.dropna(inplace=True)

In [18]:
df.head()

,time,mid_o,mid_h,mid_l,mid_c,TP,MA_TP,STD_TP,BB_UPPER,BB_LOWER,BB_SIGNAL,EMA_LONG,EMA_SHORT,MACD,SIGNAL,HIST
25,2018-01-08 02:00:00+00:00,1.35796,1.35835,1.35524,1.35558,1.356390,1.355492,0.002590,1.360673,1.350312,0,1.355542,1.355792,0.000249,0.000249,0.000000
26,2018-01-08 06:00:00+00:00,1.35558,1.35588,1.35258,1.35279,1.353750,1.355190,0.002409,1.360007,1.350373,0,1.355309,1.355325,0.000015,0.000119,-0.000104
27,2018-01-08 10:00:00+00:00,1.35280,1.35548,1.35229,1.35432,1.354030,1.354886,0.002121,1.359128,1.350644,0,1.355227,1.355169,-0.000058,0.000047,-0.000105
28,2018-01-08 14:00:00+00:00,1.35438,1.35847,1.35402,1.35593,1.356140,1.354723,0.001865,1.358453,1.350993,0,1.355285,1.355287,0.000002,0.000032,-0.000030
29,2018-01-08 18:00:00+00:00,1.35592,1.35692,1.35578,1.35661,1.356437,1.354695,0.001834,1.358362,1.351028,0,1.355394,1.355492,0.000098,0.000051,0.000047


In [19]:
df_plot = df.iloc[1000:1200]
fig = go.Figure()

fig.add_trace(go.Scatter( 
    x=df_plot.time,
    y=df_plot.SIGNAL,
    line=dict(width=2,color="#03adfc"),
    line_shape='spline',
    name='SIGNAL',
    mode='lines'
    ))        
fig.add_trace(go.Scatter( 
    x=df_plot.time,
    y=df_plot.MACD,
    line=dict(width=2,color="#fc4a03"),
    line_shape='spline',
    name='MACD',
    mode='lines'
    ))         
fig.add_trace(go.Bar( 
    x=df_plot.time,
    y=df_plot.HIST,
    name='HIST'
    ))         

fig.update_layout(width=1000,height=400,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False),
    rangebreaks=[
        dict(bounds=["sat", "mon"])
    ]
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()